In [ ]:
! pip install requests fpdf fpdf2

## Added Links + Sorting based on Timestamp

In [ ]:
LEETCODE_SESSION = "YOUR_LEETCODE_SESSION_HERE"  # Replace with your actual session cookie
CSRF_TOKEN = "YOUR_CSRF_TOKEN_HERE"  # Replace with your actual CSRF token
USERNAME = "YOUR_USERNAME_HERE"  # Replace with your LeetCode username
import requests
import json
import re
from fpdf import FPDF
from datetime import datetime

# ---------------- CONFIG ----------------


HEADERS = {
    "cookie": f"LEETCODE_SESSION={LEETCODE_SESSION}; csrftoken={CSRF_TOKEN};",
    "x-csrftoken": CSRF_TOKEN,
    "referer": "https://leetcode.com",
    "Content-Type": "application/json"
}

PRIMARY_COLOR = (250, 166, 26)
SECONDARY_COLOR = (40, 40, 40)
ACCENT_COLORS = {"Easy": (0, 184, 163), "Medium": (255, 192, 30), "Hard": (255, 55, 95)}
FONT_REGULAR = "DejaVuSans.ttf"
FONT_BOLD = "DejaVuSans-Bold.ttf"

# ---------------- SCRAPE SUBMISSION HISTORY ----------------
def scrape_submission_history():
    submissions = {}
    page = 0
    while True:
        url = f"https://leetcode.com/api/submissions/?offset={page*20}&limit=20"
        r = requests.get(url, headers=HEADERS)
        data = r.json()
        if not data.get("submissions_dump"):
            break
        for sub in data["submissions_dump"]:
            slug = sub["title_slug"]
            ts = int(sub["timestamp"])
            if slug not in submissions or ts > submissions[slug]:
                submissions[slug] = ts
        page += 1
    return submissions

# ---------------- PDF CLASS ----------------
class PDF(FPDF):
    def header(self):
        self.set_fill_color(*PRIMARY_COLOR)
        self.rect(0, 0, self.w, 20, style='F')
        self.set_y(10)
        self.set_font('DejaVuSans-Bold', '', 16)
        self.set_text_color(255, 255, 255)
        self.cell(0, 10, 'LeetCode Solved Questions Report', align='C')
        self.set_text_color(*SECONDARY_COLOR)
        self.ln(10)  # <-- add this to ensure content starts below header

    def footer(self):
        self.set_y(-15)
        self.set_font('DejaVuSans', '', 10)
        self.set_text_color(150, 150, 150)
        self.cell(0, 10, f'Page {self.page_no()}/{{nb}}', 0, 0, 'C')

# ---------------- PDF REPORT ----------------
def create_pdf(solved_problems):
    pdf = PDF()
    pdf.add_font('DejaVuSans', '', FONT_REGULAR)
    pdf.add_font('DejaVuSans-Bold', '', FONT_BOLD)
    pdf.set_title('LeetCode Solved Report')
    pdf.set_author(USERNAME)
    pdf.alias_nb_pages()

    # Cover page
    pdf.add_page()
    pdf.set_font('DejaVuSans-Bold', '', 24)
    pdf.set_text_color(*PRIMARY_COLOR)
    pdf.cell(0, 60, '', ln=True)
    pdf.cell(0, 15, 'LeetCode Full Solved Report', ln=True, align='C')
    pdf.set_font('DejaVuSans', '', 16)
    pdf.set_text_color(*SECONDARY_COLOR)
    pdf.cell(0, 10, f'Generated for: {USERNAME}', ln=True, align='C')
    today = datetime.today().strftime('%B %d, %Y')
    pdf.set_font('DejaVuSans', '', 14)
    pdf.set_text_color(150, 150, 150)
    pdf.cell(0, 10, today, ln=True, align='C')
    pdf.cell(0, 20, '', ln=True)

    # Summary stats
    counts = {
        "Easy": sum(1 for p in solved_problems if p["difficulty"] == "Easy"),
        "Medium": sum(1 for p in solved_problems if p["difficulty"] == "Medium"),
        "Hard": sum(1 for p in solved_problems if p["difficulty"] == "Hard"),
    }
    pdf.set_font('DejaVuSans-Bold', '', 16)
    pdf.set_text_color(*PRIMARY_COLOR)
    pdf.cell(0, 10, "Solved Problems Summary", ln=True)
    pdf.ln(5)
    pdf.set_font('DejaVuSans', '', 14)
    pdf.set_text_color(*SECONDARY_COLOR)
    pdf.cell(0, 8, f"Total Solved: {len(solved_problems)}", ln=True)
    pdf.cell(0, 8, f"Easy: {counts['Easy']} | Medium: {counts['Medium']} | Hard: {counts['Hard']}", ln=True)

    # Table of contents
    pdf.add_page()
    pdf.set_font('DejaVuSans-Bold', '', 16)
    pdf.set_text_color(*PRIMARY_COLOR)
    pdf.cell(0, 10, 'All Solved Problems (latest first)', ln=True)
    pdf.ln(5)
    pdf.set_font('DejaVuSans', '', 12)
    for i, q in enumerate(solved_problems, start=1):
        color = ACCENT_COLORS.get(q["difficulty"], SECONDARY_COLOR)
        pdf.set_text_color(*color)
        link = f"https://leetcode.com/problems/{q['titleSlug']}/"
        pdf.cell(0, 7, f"{i}. {q['title']} [{q['difficulty']}]", ln=True, link=link)
    output_path = "leetcode_full_report.pdf"
    pdf.output(output_path)
    print(f"\n✅ Report generated: {output_path} with {len(solved_problems)} problems.")

# ---------------- MAIN ----------------
def main():
    print("🔍 Scraping submission history...")
    submissions = scrape_submission_history()
    print("📊 Fetching all solved problems metadata...")
    data = requests.get("https://leetcode.com/api/problems/all/", headers=HEADERS).json()
    solved = []
    for p in data["stat_status_pairs"]:
        if p.get("status") == "ac":
            slug = p["stat"]["question__title_slug"]
            solved.append({
                "titleSlug": slug,
                "title": p["stat"]["question__title"],
                "difficulty": ["Easy", "Medium", "Hard"][p["difficulty"]["level"] - 1],
                "timestamp": submissions.get(slug, 0)
            })
    solved.sort(key=lambda x: x["timestamp"], reverse=True)
    print(f"✅ You have solved {len(solved)} problems.")
    create_pdf(solved)

if __name__ == "__main__":
    main()


🔍 Scraping submission history...
📊 Fetching all solved problems metadata...
✅ You have solved 106 problems.

✅ Report generated: leetcode_full_report.pdf with 106 problems.


/tmp/ipython-input-26-3819755603.py:75: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 60, '', ln=True)
/tmp/ipython-input-26-3819755603.py:76: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 15, 'LeetCode Full Solved Report', ln=True, align='C')
/tmp/ipython-input-26-3819755603.py:79: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Generated for: {USERNAME}', ln=True, align='C')
/tmp/ipython-input-26-3819755603.py:83: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, today, ln=True, align='C')
/tmp/ipython-input-26-3819755603.py:84: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of l

## Only Question Names and their level

In [ ]:
# 🔧 YOUR COOKIES
LEETCODE_SESSION = "eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJfYXV0aF91c2VyX2lkIjoiMTE4ODUwMjAiLCJfYXV0aF91c2VyX2JhY2tlbmQiOiJhbGxhdXRoLmFjY291bnQuYXV0aF9iYWNrZW5kcy5BdXRoZW50aWNhdGlvbkJhY2tlbmQiLCJfYXV0aF91c2VyX2hhc2giOiI4YjYzNmY1NDA0MDk5NTgzMjQ2YzBmNDhlNzU1ODgzZDQ4ZjU2NDQ4ZjBlMWNjMTI5YzBmYWFhZTdkMDdhOGQ0Iiwic2Vzc2lvbl91dWlkIjoiOWY2ZDFmY2IiLCJpZCI6MTE4ODUwMjAsImVtYWlsIjoia3Jpc2huYW12YS5idGVjaDIyQHJ2dS5lZHUuaW4iLCJ1c2VybmFtZSI6InVzZXI4MTkzU3EiLCJ1c2VyX3NsdWciOiJ1c2VyODE5M1NxIiwiYXZhdGFyIjoiaHR0cHM6Ly9hc3NldHMubGVldGNvZGUuY29tL3VzZXJzL2F2YXRhcnMvYXZhdGFyXzE3MDM5Mzk1NTAucG5nIiwicmVmcmVzaGVkX2F0IjoxNzUxNDcyNjM0LCJpcCI6IjEyMi4xNzIuODQuMTg4IiwiaWRlbnRpdHkiOiIwZmU2ZmViNTQyODlmNGM2NzAyN2VjMDZjYzIxMzFmOCIsImRldmljZV93aXRoX2lwIjpbIjI5NmVkNGMzOWYwMjNmNTNiNTU1NDEyZGY4YzcyZjM2IiwiMTIyLjE3Mi44NC4xODgiXX0.vd7ZwIIuunQDUs7d0oCrjyJ202uPFk9buGIBKWavDVI"
CSRF_TOKEN = "FX1QrteAEHvRM0bO3aFLXVUWHen65HdWSlnYOdxwF42zoAx8yHSTFTEVkH9o2bEr"
USERNAME = "user8193Sq"

import requests
import json
import re
import os
from fpdf import FPDF
from datetime import datetime

# -------------------------------
# CONFIGURATION
# -------------------------------

HEADERS = {
    "cookie": f"LEETCODE_SESSION={LEETCODE_SESSION}; csrftoken={CSRF_TOKEN};",
    "x-csrftoken": CSRF_TOKEN,
    "referer": "https://leetcode.com",
    "Content-Type": "application/json"
}

PRIMARY_COLOR = (250, 166, 26)
SECONDARY_COLOR = (40, 40, 40)
ACCENT_COLORS = {
    "Easy": (0, 184, 163),
    "Medium": (255, 192, 30),
    "Hard": (255, 55, 95)
}

FONT_REGULAR = "DejaVuSans.ttf"
FONT_BOLD = "DejaVuSans-Bold.ttf"

# -------------------------------
# FETCH SOLVED PROBLEMS
# -------------------------------

def get_all_solved_problems():
    url = "https://leetcode.com/api/problems/all/"
    r = requests.get(url, headers=HEADERS)
    data = r.json()
    solved = []
    for p in data["stat_status_pairs"]:
        if p["status"] == "ac":
            solved.append({
                "questionFrontendId": p["stat"]["frontend_question_id"],
                "title": p["stat"]["question__title"],
                "difficulty": ["Easy","Medium","Hard"][p["difficulty"]["level"]-1]
            })
    return solved

# -------------------------------
# PDF GENERATOR
# -------------------------------

class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.difficulty_colors = ACCENT_COLORS
        self.primary_color = PRIMARY_COLOR
        self.secondary_color = SECONDARY_COLOR
        self.set_auto_page_break(auto=True, margin=15)
        self.set_margins(15, 20, 15)

    def header(self):
        self.set_fill_color(*self.primary_color)
        self.rect(0, 0, self.w, 20, style='F')
        self.set_y(10)
        self.set_font('DejaVuSans-Bold', '', 16)
        self.set_text_color(255, 255, 255)
        self.cell(0, 10, 'LeetCode Solutions Report', ln=True, align='C')
        self.set_text_color(*self.secondary_color)

    def footer(self):
        self.set_y(-15)
        self.set_font('DejaVuSans', '', 10)
        self.set_text_color(150, 150, 150)
        self.cell(0, 10, f'Page {self.page_no()}/{{nb}}', 0, 0, 'C')

# -------------------------------
# CREATE PDF REPORT
# -------------------------------

def create_pdf(solved_problems):
    pdf = PDF()
    pdf.add_font('DejaVuSans', '', FONT_REGULAR, uni=True)
    pdf.add_font('DejaVuSans-Bold', '', FONT_BOLD, uni=True)
    pdf.set_title('LeetCode Solutions Report')
    pdf.set_author(USERNAME)
    pdf.alias_nb_pages()

    # Cover page
    pdf.add_page()
    pdf.set_font('DejaVuSans-Bold', '', 24)
    pdf.set_text_color(*PRIMARY_COLOR)
    pdf.cell(0, 60, '', ln=True)
    pdf.cell(0, 15, 'LeetCode Solutions Report', ln=True, align='C')
    pdf.set_font('DejaVuSans', '', 16)
    pdf.set_text_color(*SECONDARY_COLOR)
    pdf.cell(0, 10, f'Generated for: {USERNAME}', ln=True, align='C')
    today = datetime.today().strftime('%B %d, %Y')
    pdf.set_font('DejaVuSans', '', 14)
    pdf.set_text_color(150, 150, 150)
    pdf.cell(0, 10, today, ln=True, align='C')
    pdf.cell(0, 30, '', ln=True)

    # Stats
    pdf.set_fill_color(*PRIMARY_COLOR)
    pdf.set_text_color(255, 255, 255)
    pdf.set_font('DejaVuSans-Bold', '', 16)
    pdf.cell(0, 10, ' Summary Statistics ', ln=True, align='C', fill=True)
    pdf.ln(10)

    counts = {
        "Easy": sum(1 for p in solved_problems if p["difficulty"] == "Easy"),
        "Medium": sum(1 for p in solved_problems if p["difficulty"] == "Medium"),
        "Hard": sum(1 for p in solved_problems if p["difficulty"] == "Hard")
    }
    pdf.set_font('DejaVuSans', '', 14)
    pdf.set_text_color(*SECONDARY_COLOR)
    pdf.cell(0, 8, f"Total Solved Problems: {len(solved_problems)}", ln=True)
    for diff, color in ACCENT_COLORS.items():
        pdf.set_text_color(*color)
        pdf.cell(0, 8, f"{diff}: {counts[diff]} problems", ln=True)

    # Table of Contents
    pdf.add_page()
    pdf.set_font('DejaVuSans-Bold', '', 16)
    pdf.set_text_color(*PRIMARY_COLOR)
    pdf.cell(0, 10, 'Table of Contents', ln=True)
    pdf.ln(5)
    pdf.set_font('DejaVuSans', '', 12)
    pdf.set_text_color(*SECONDARY_COLOR)
    for i, q in enumerate(solved_problems, start=1):
        color = ACCENT_COLORS[q["difficulty"]]
        pdf.set_text_color(*color)
        toc_line = f"{i}. {q['questionFrontendId']} - {q['title']} [{q['difficulty']}]"
        pdf.cell(0, 7, toc_line, ln=True)

    # Save
    output_path = "leetcode_report.pdf"
    pdf.output(output_path)
    print(f"\n✅ Report generated: {output_path} with {len(solved_problems)} questions.")
    return output_path

# -------------------------------
# MAIN
# -------------------------------

def main():
    print("Fetching solved questions...")
    solved = get_all_solved_problems()
    print(f"Found {len(solved)} solved problems.")
    create_pdf(solved)

if __name__ == "__main__":
    main()


Fetching solved questions...
Found 106 solved problems.

✅ Report generated: leetcode_report.pdf with 106 questions.


/tmp/ipython-input-15-3366188832.py:87: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVuSans', '', FONT_REGULAR, uni=True)
/tmp/ipython-input-15-3366188832.py:88: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVuSans-Bold', '', FONT_BOLD, uni=True)
/tmp/ipython-input-15-3366188832.py:72: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  self.cell(0, 10, 'LeetCode Solutions Report', ln=True, align='C')
/tmp/ipython-input-15-3366188832.py:97: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 60, '', ln=True)
/tmp/ipython-input-15-3366188832.py:98: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0